In [1]:
#!/usr/bin/env python2.7 
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
#使用tensorflow自带的工具加载MNIST手写数字集合
#mnist = input_data.read_data_sets('/data/mnist', one_hot=True)

In [3]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [4]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [5]:
graph = tf.Graph()
with graph.as_default():
    
    serialized_tf_example = tf.placeholder(tf.string, name='tf_example') 
    feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
    tf_example = tf.parse_example(serialized_tf_example, feature_configs)
    x = tf.identity(tf_example['x'], name='x')  # use tf.identity() to assign name
    
    #x = tf.placeholder(tf.float32, [None, 784])
    y_= tf.placeholder(tf.float32,[None,10])
    w = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    W_fc1 = weight_variable([7 * 7 * 64, 1024])
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    values, indices = tf.nn.top_k(y_conv, 10) 
    table = tf.contrib.lookup.index_to_string_table_from_tensor( tf.constant([str(i) for i in xrange(10)]))
    prediction_classes = table.lookup(tf.to_int64(indices))
    
    cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [6]:
session = tf.InteractiveSession(graph=graph)
model_dir = "/data/1xiu/mnist_test"
model_name = "mnist_models"
model_version="5"
def loadModel():
    # 恢复模型
    saver.restore(session, os.path.join(model_dir, model_name))
    print("恢复模型成功！")
    
    # Export model
    # WARNING(break-tutorial-inline-code): The following code snippet is
    # in-lined in tutorials, please update tutorial documents accordingly
    # whenever code changes.
    #export_path_base = "/data/1xiu/mnist_test"
    export_path = os.path.join(
      tf.compat.as_bytes(model_dir),
      tf.compat.as_bytes(model_version))
    print("Exploting trained modek to ", export_path)
    
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)
#
   # # Build the signature_def_map.
    classification_inputs = tf.saved_model.utils.build_tensor_info(
      serialized_tf_example)
    classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
      prediction_classes)
    classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)
    
    classification_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={
             tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                  classification_inputs
          },
          outputs={
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                  classification_outputs_classes,
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                  classification_outputs_scores
          },
          method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))
    tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
    tensor_info_y = tf.saved_model.utils.build_tensor_info(y_conv)
#
    prediction_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
         inputs={'images': tensor_info_x},
          outputs={'scores': tensor_info_y},
          method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))
    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    builder.add_meta_graph_and_variables(
      session, [tf.saved_model.tag_constants.SERVING],
      signature_def_map={
          'predict_images':
              prediction_signature,
          tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
              classification_signature,
      },
      legacy_init_op=legacy_init_op)

    builder.save()

loadModel()

INFO:tensorflow:Restoring parameters from /data/1xiu/mnist_test/mnist_models
恢复模型成功！
('Exploting trained modek to ', '/data/1xiu/mnist_test/5')


AssertionError: Export directory already exists. Please specify a different export directory: /data/1xiu/mnist_test/5